In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
import datetime
import os

C:\Users\Yuriy Podmogaev\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Downloading data
Shipments - combined data set of shipments grains and feedstaff from Russia from 2013 till 2022 years

In [2]:
# collecting few excel files
path = "C:/Users/Yuriy Podmogaev/Desktop/Коробка/Freight/data/TBI"
sets_lists = os.listdir(path)

shipments = pd.DataFrame()

for i in range(0,len(sets_lists)):
    df = pd.read_excel(str(f'C:/Users/Yuriy Podmogaev/Desktop/Коробка/Freight/data/TBI/{sets_lists[i]}'))
    shipments = pd.concat([shipments, df])

shipments['Loading Completed'] = pd.to_datetime(shipments['Loading Completed'])

## Preparing data

In [3]:
# cut off auto and rail shipments
#shipments = shipments[(shipments.Port !='AUTO-ROAD') & (shipments.Port != 'RAILWAY') & (shipments.Cargo != 'TOTAL')]

# cut off conteiner shipments using qtty
#shipments = shipments[shipments.Quantity >= 2000]

# cut off inland trading
shipments = shipments[shipments.Destination != 'RUSSIA']

# cuting off movements to transshipment area to avoid double counting
shipments = shipments[shipments.Transhipment.isnull()]

# cutting of 'tottal' lines
shipments = shipments[shipments.Cargo != 'TOTAL']

# format date
shipments['Year'] = pd.DatetimeIndex(shipments['Loading Completed']).year

# drop columns
drop_col = ['Terminal', 'Berth / Facility', 'IMO', 'Agent', 'Flag']
shipments = shipments.drop(drop_col, axis=1)

In [4]:
shipments[:3]

,Port,Vessel,Cargo,Quantity,Loading Completed,Shipper,Destination,PortDischarge,BuyerName,Transhipment,Year
0,AUTO-ROAD,AUTOTRUCKS,SOYA BEAN MEAL,49.00,2019-10-25,TRADE IMPEX,UKRAINE,DONETSK / UA DOK,DON TRADE IMPEX,NaN,2019
1,AUTO-ROAD,AUTOTRUCKS,SOYA BEAN MEAL,48.00,2019-09-25,TRADE IMPEX,UKRAINE,DONETSK / UA DOK,DON TRADE IMPEX,NaN,2019
2,AUTO-ROAD,AUTOTRUCKS,CORN GLUTEN,20.01,2017-03-21,LUGAN',UKRAINE,NaN,LUGANSK-NIVA,NaN,2017


In [5]:
# grouping commodity type
WHEAT = ['MILLING WHEAT', 'FEED WHEAT', 'DURUM WHEAT', 'KAZAKH DURUM WHEAT', 
         '4 GR SOFT WHEAT', '3 GR SOFT WHEAT', '2 GR SOFT WHEAT', 'GRAINS', 
         'EMMER WHEAT']

# CORN & BARLEY remain as is

FEED = ['RAPESEED MEAL', 'SOYA BEAN MEAL', 'SUGAR BEET PULP',
       'CAMELINA MEAL', 'SOYA HULL PELLETS', 'SUNFLOWERSEED MEAL',
       'SOYA PROTEIN', 'SOYA BEAN HULL', 'FLAXSEED MEAL',
       'SAFFLOWER SEED MEAL', 'WHEAT BRAN', 'SUNFLOWERSEED HULL',
       'SOYA BEAN CAKE', 'OTHER MEAL', 'CORN BRAN', 'CORN GLUTEN', 
       'SUNFLOWERSEED CAKE', 'RICE BRAN', 'FLAXSEED CAKE', 'CORN GERM',
       'BUCKWHEAT HULL', 'MILK THISTLE MEAL', 'CEDAR CAKE', 'RAPESEED CAKE',
       'CEDAR MEAL', 'CORN CAKE', 'WHEAT GLUTEN', 'MUSTARD BRAN']

OILSEED = ['MOLASSES', 'BARDA', 'FLAXSEED', 'OILSEED', 'SOYA BEAN', 
           'MILLETSEED', 'RAPESEED', 'SUNFLOWER', 'KAZAKH FLAXSEED', 
           'KAZAKH RAPESEED','PIGEON BEAN', 'BEAN', 'OILSEED', 'HARICOT BEAN']

OTHER = ['YEAST', 'RYE', 'PEAS', 'CHICKPEAS', 'LENTIL', 'OAT', 'SORGHUM', 
         'SAFFLOWER', 'WHEAT FLOUR', 'RICE', 'KAZAKH LENTIL', 'MUSTARD', 
         'CAMELINA', 'BREWING WASTE', 'OTHER', 'BUCKWHEAT', 'RADISH SEED', 
         'PUMPKIN SEED', 'COMMON VETCH', 'MILK THISTLE', 'TRITIKALE', 'HEMPSEED',
         'CORIANDER', 'MALT BARLEY', 'SOYBEAN FLOUR', 'SESAME', 'LUPINE']

shipments['Com_type'] = shipments['Cargo'].replace(WHEAT, 'WHEAT').replace(FEED, 'FEED').replace(OILSEED, 'OILSEED') \
                        .replace(OTHER, 'OTHER')

In [6]:
shipments[:3]

,Port,Vessel,Cargo,Quantity,Loading Completed,Shipper,Destination,PortDischarge,BuyerName,Transhipment,Year,Com_type
0,AUTO-ROAD,AUTOTRUCKS,SOYA BEAN MEAL,49.00,2019-10-25,TRADE IMPEX,UKRAINE,DONETSK / UA DOK,DON TRADE IMPEX,NaN,2019,FEED
1,AUTO-ROAD,AUTOTRUCKS,SOYA BEAN MEAL,48.00,2019-09-25,TRADE IMPEX,UKRAINE,DONETSK / UA DOK,DON TRADE IMPEX,NaN,2019,FEED
2,AUTO-ROAD,AUTOTRUCKS,CORN GLUTEN,20.01,2017-03-21,LUGAN',UKRAINE,NaN,LUGANSK-NIVA,NaN,2017,FEED


In [7]:
# cleaning Shippers names all after ' / '
shipments['Shipper'] = shipments['Shipper'].astype(str)

shipments['Shipper'].astype(str)
def clean_string(s):
    if ' / ' in s:
        return s.split(' / ')[0]
    else:
        return s
    
shipments['Shipper'] = shipments['Shipper'].apply(clean_string)

In [11]:
# Combine abt same names
def merge_names(shipments):
    unique_names = shipments['Shipper'].unique()
    merged_names = []
    for name1 in unique_names:
        already_merged = False
        for name2 in merged_names:
            if fuzz.ratio(name1, name2) >= 90:
                shipments.loc[shipments['Shipper'] == name1, 'Shipper'] = name2
                already_merged = True
                break
        if not already_merged:
            merged_names.append(name1)
    return shipments

shipments = merge_names(shipments)

In [12]:
shipments[:3]

,Port,Vessel,Cargo,Quantity,Loading Completed,Shipper,Destination,PortDischarge,BuyerName,Transhipment,Year,Com_type
0,AUTO-ROAD,AUTOTRUCKS,SOYA BEAN MEAL,49.00,2019-10-25,TRADE IMPEX,UKRAINE,DONETSK / UA DOK,DON TRADE IMPEX,NaN,2019,FEED
1,AUTO-ROAD,AUTOTRUCKS,SOYA BEAN MEAL,48.00,2019-09-25,TRADE IMPEX,UKRAINE,DONETSK / UA DOK,DON TRADE IMPEX,NaN,2019,FEED
2,AUTO-ROAD,AUTOTRUCKS,CORN GLUTEN,20.01,2017-03-21,LUGAN',UKRAINE,NaN,LUGANSK-NIVA,NaN,2017,FEED


In [13]:
# Segmentation by type of shipments

# quantites to separate deepsea and shortsea shipments
qmax = 15000
qmin = 1000

# Segmentation
shipments.loc[shipments.Quantity >= qmax, 'Type_of_shipm'] = 'Deepsea'
shipments.loc[(shipments.Quantity <= qmax) & (shipments.Quantity >= qmin), 'Type_of_shipm'] = 'Shortsea'
shipments.loc[(shipments.Vessel == 'AUTOTRUCKS') | (shipments.Port == 'AUTO-ROAD'), 'Type_of_shipm'] = 'Auto'
shipments.loc[(shipments.Vessel == 'WAGONS') | (shipments.Port == 'RAILWAY') , 'Type_of_shipm'] = 'Rail'
shipments.loc[shipments.Vessel == 'CONTAINERS' , 'Type_of_shipm'] = 'FLC'
shipments.loc[shipments.Type_of_shipm.isna() & shipments.Vessel.notna(), 'Type_of_shipm'] = 'Shortsea'


In [14]:
# Segmentation by geography of shipments

ASIA = [asi.upper() for asi in pd.read_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Agro/Regions_counties/Asia.xlsx').Country.to_list()]
AFRICA = [asi.upper() for asi in pd.read_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Agro/Regions_counties/Africa.xlsx').Country.to_list()]
EUROPE = [asi.upper() for asi in pd.read_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Agro/Regions_counties/Europe.xlsx').Country.to_list()]
LATIN = [asi.upper() for asi in pd.read_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Agro/Regions_counties/Latin America and the Caribbean.xlsx').Country.to_list()]
N_AMERICA = [asi.upper() for asi in pd.read_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Agro/Regions_counties/Northern America.xlsx').Country.to_list()]
OCEANIA = [asi.upper() for asi in pd.read_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Agro/Regions_counties/Oceania.xlsx').Country.to_list()]

shipments['Region'] = shipments['Destination'].replace(ASIA, 'Asia').replace(AFRICA, 'Africa').replace(EUROPE, 'Europe') \
                        .replace(LATIN, 'Latin_Am').replace(N_AMERICA, 'North_Am').replace(OCEANIA, 'Oceania')

In [15]:
shipments.to_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Freight/data/combined.xlsx', index=False)

In [105]:
# preparing dataset for counting n-share of market
n = 0.90  
df = shipments[['Quantity', 'Shipper', 'Year' ]][shipments.Year >= 2018]

# make dataframe counting 'Quantity' for yeach 'Shipper' per 'Year'
df_count = df.groupby(['Year','Shipper']).agg({'Quantity':'sum'}).reset_index()

# count 'Shipper' covering n sum of 'Quantity' for every  'Year'
df_result = pd.DataFrame(columns=['Year', 'Shipper_count'])
for year in df_count['Year'].unique():
    df_year = df_count[df_count['Year'] == year]
    df_year = df_year.sort_values(by='Quantity', ascending=False)
    df_year['Cumulative_Percentage'] = df_year['Quantity'].cumsum() / df_year['Quantity'].sum()
    count = df_year[df_year['Cumulative_Percentage'] <= n]['Shipper'].nunique()
    row = {'Year': year, 'Shipper_count': count}
    df_result = df_result.append(row, ignore_index=True)
    
# saving result
df_result.to_excel('C:/Users/Yuriy Podmogaev/Desktop/Коробка/Freight/data/market_share.xlsx', index=False)


In [106]:
df_result

,Year,Shipper_count
0,2018,100
1,2019,117
2,2020,89
3,2021,86
4,2022,75


In [27]:
df = pd.DataFrame({
    'Quantity': [32, 25, 47, 29, 18, 4, 2, 1, 42, 9, 8, 4, 2, 3],
    'Shipper': ['Sarah','John', 'Mary', 'David', 'Sarah', 'Peter', 'Daviвd', 'Sвarah', 'Petвer', 'John', 'Mary', 'David', 'Sarah', 'Peter'],
    'Year': [2012, 2014]*7
})

In [28]:
df

,Quantity,Shipper,Year
0,32,Sarah,2012
1,25,John,2014
2,47,Mary,2012
3,29,David,2014
4,18,Sarah,2012
5,4,Peter,2014
6,2,Daviвd,2012
7,1,Sвarah,2014
8,42,Petвer,2012
9,9,John,2014


In [ ]:
a = shipments[shipments.Quantity >= 9000].groupby('Year')['Shipper'].nunique()

In [ ]:
a[a.index >= 2018].plot(kind = 'bar', ylim=(30,1500))

In [ ]:
tz_counts = shipments[shipments.Year == 2018].groupby('Shipper')['Quantity'].sum().sort_values(ascending=False)

In [ ]:
import seaborn as sns
subset = tz_counts[:15]
sns.barplot(y=subset.index, x=subset.values)

In [ ]:
tz_counts = shipments[shipments.Year == 2022].groupby('Shipper')['Quantity'].sum().sort_values(ascending=False)
import seaborn as sns
subset = tz_counts[:15]
sns.barplot(y=subset.index, x=subset.values)

In [ ]:
tz_counts = shipments[shipments.Year == 2018].groupby('Shipper')['Quantity'].sum().sort_values(ascending=False)
tz_counts[:100].sum()/tz_counts[:].sum()

In [ ]:
tz_counts = shipments[shipments.Year == 2022].groupby('Shipper')['Quantity'].sum().sort_values(ascending=False)
tz_counts[:100].sum()/tz_counts[:].sum()

In [ ]:
shipments.info()

In [9]:
df = pd.DataFrame({
    'Name': ['John', 'Mary', 'David', 'Sarah', 'Peter', 'Daviвd', 'Sвarah', 'Petвer'],
    'Age': [32, 25, 47, 29, 18, 4, 2, 1],
})

In [10]:
df

,Name,Age
0,John,32
1,Mary,25
2,David,47
3,Sarah,29
4,Peter,18
5,Daviвd,4
6,Sвarah,2
7,Petвer,1


In [ ]:
# sort the dataframe by age in descending order
df_sorted = df.sort_values(by='Age', ascending=False)

# calculate the cumulative sum of ages
cumulative_sum = df_sorted['Age'].cumsum()

# calculate the percentage of the total sum of ages for each row
percentages = cumulative_sum / df_sorted['Age'].sum()

# find the index of the first row that covers at least 95% of the total sum of ages
index_95 = (percentages >= 0.95).idxmax()

# count the number of names in the first rows up to the index_95
count_names = len(df_sorted.iloc[:index_95, :])

print(count_names)